In [10]:
## import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [11]:
## Read the csv file
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
## Preprocess the data
## Drop the columns which are not required
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [13]:
## Encode  the categorical columns
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [14]:
# Onehot encode the Geography column
onehot_encoder_geo =OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data['Geography'].values.reshape(-1,1)).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [15]:
## Combine the geo_encoded_df with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()    

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
## split the data into independent and dependent variables
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [19]:
## Split the data into training and testing data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Standardize the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [20]:
## Saving all the pickle files
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Regression Implementation   

In [22]:
## import Tensorflow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [27]:
## Build Regression ANN model
model=Sequential([
    Dense(units=64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1
    Dense(units=32,activation='relu'), ## HL2
    Dense(units=1)]) ## Output layer with 1 neuron with activation function as default is linear activation function for regression which means whatever output I am getting , I am using the same 



In [30]:
## compile the model for forward and backward propagation to update the weights
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
## Setting up the early stopping and the tensorboard callback for exit the training when the model is not learning and visualisation of logs data

from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime

log_dir='regressionlog/fit/'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
TensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [33]:
## Train the ANN Regression model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[TensorBoard_callback,early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100372.4766 - mean_absolute_error: 100372.4766 - val_loss: 98489.6406 - val_mean_absolute_error: 98489.6406
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99468.4688 - mean_absolute_error: 99468.4688 - val_loss: 96628.0312 - val_mean_absolute_error: 96628.0312
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 96219.5547 - mean_absolute_error: 96219.5547 - val_loss: 91873.2500 - val_mean_absolute_error: 91873.2500
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 89879.1016 - mean_absolute_error: 89879.1016 - val_loss: 84106.2188 - val_mean_absolute_error: 84106.2188
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 80904.5078 - mean_absolute_error: 80904.5078 - val_loss: 74566.0156 - val_mean_absolute_error: 74566.0156
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 70928.4375 - mea

In [ ]:
## Load tensorboard ext
%load_ext tensorboard
%tensorboard --logdir regressionlog/fit

In [40]:
## Evaluate the model on test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test Loss: {test_loss} and Test MAE: {test_mae}')

63/63 [==============================] - 0s 1ms/step - loss: 50196.9805 - mean_absolute_error: 50196.9805
Test Loss: 50196.98046875 and Test MAE: 50196.98046875


In [41]:
## Save my regression model
model.save('regression_model.h5')

c:\Users\Ankit Ghai\Documents\PycharmProjects\pythonProject\ANNClassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
